In [6]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    Doc
)
import re
import json

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

pos_mapping = {
    "nsubj": "Подлежащее",
    "root": "Сказуемое",
    "obj": "Прямое дополнение",
    "obl": "Обстоятельство",
    "amod": "Согласованное определение",
    "case": "Предлог",
    "cc": "Соединительный союз",
    "conj": "Сочинительная связь",
    "advcl": "Обстоятельственное придаточное предложение",
    "det": "Определитель",
    "punct": "Знак пунктуации",
    "aux": "Вспомогательный глагол",
    "advmod": "Обстоятельственное наречие",
    "xcomp": "Дополнительный член предложения",
    "nummod": "Количественное определение",
    "nmod": "Несогласованное определение",
    "parataxis": "Паратаксис",
    "nummod:gov": "Количественное определение (управляющее слово)",
    "acl": "Определительное придаточное предложение",
    "iobj": "Косвенное дополнение",
    "appos": "Приложение"
}

feat_mapping = {
   "Case": {
                "Nom": "Именительный падеж",
                "Gen": "Родительный падеж",
                "Dat": "Дательный падеж",
                "Acc": "Винительный падеж",
                "Ins": "Творительный падеж",
                "Loc": "Предложный падеж"
            },
            "Number": {
                "Sing": "Единственное число",
                "Plur": "Множественное число"
            },
            "Gender": {
                "Masc": "Мужской род",
                "Fem": "Женский род",
                "Neut": "Средний род"
            },
            "Tense": {
                "Past": "Прошедшее время",
                "Pres": "Настоящее время",
                "Fut": "Будущее время"
            },
            "Aspect": {
                "Imp": "Несовершенный вид",
                "Perf": "Совершенный вид"
            },
            "Mood": {
                "Ind": "Изъявительное наклонение",
                "Imp": "Повелительное наклонение"
            },
            "VerbForm": {
                "Fin": "Личная форма",
                "Inf": "Инфинитив",
                "Part": "Причастие",
                "Ger": "Деепричастие"
            },
            "Person": {
                "1": "1-е лицо",
                "2": "2-е лицо",
                "3": "3-е лицо"
            },
            "Animacy": {
                "Anim": "Одушевленный",
                "Inan": "Неодушевленный"
            },
            "Voice": {
                "Act": "Действительный залог",
                "Pass": "Страдательный залог",
                "Mid": "Средний залог"
            },
            "Degree": {
                "Pos": "Положительная степень",
                "Cmp": "Сравнительная степень",
                "Sup": "Превосходная степень"
            },
            "Polarity": {
                "Neg": "Отрицательная полярность"
            }
}

def is_russian(word):
    return bool(re.match(r'^[а-яА-ЯёЁ-]+$', word))

def parse_grammar_feats(feats_dict):
    grammar = {}
    if not feats_dict:
        return grammar
    for key, val in feats_dict.items():
        if key in feat_mapping:
            ru_key = {
                'Case': 'падеж',
                'Number': 'число', 
                'Gender': 'род',
                'Tense': 'время'
            }.get(key, key)
            
            if isinstance(val, list):
                val = val[0]
            
            ru_val = feat_mapping[key].get(val, val)
            grammar[ru_key] = ru_val
        else:
            grammar[key] = val
    return grammar

def process_text(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    
    result = {'sentences': []}
    global_index = 0
    
    for sent in doc.sents:
        tokens = []
        for token in sent.tokens:
            token_data = {
                'index': global_index,
                'token': token.text
            }
            global_index += 1
            
            # Определение типа токена
            if token.pos == 'PUNCT':
                token_data['type'] = 'punct'
            elif re.fullmatch(r'[\d,\.]+', token.text):
                token_data['type'] = 'number'
            elif is_russian(token.text):
                token_data['type'] = 'russian'
                token_data['lemma'] = token.lemma
                pos = pos_mapping.get(token.pos, token.pos)
                feats = parse_grammar_feats(token.feats)
                grammar = {'часть речи': pos, **feats}
                token_data['grammar_categories'] = grammar
            else:
                token_data['type'] = 'foreign'
            
            tokens.append(token_data)
        result['sentences'].append({'tokens': tokens})
    
    return result

# Пример использования
text = "Привет мир! В 2023 году проект собрал 1,5 млн. просмотров. Hello world!"
result = process_text(text)
print(json.dumps(result, ensure_ascii=False, indent=2))

{
  "sentences": [
    {
      "tokens": [
        {
          "index": 0,
          "token": "Привет",
          "type": "russian",
          "lemma": "привет",
          "grammar_categories": {
            "часть речи": "X",
            "Foreign": "Yes"
          }
        },
        {
          "index": 1,
          "token": "мир",
          "type": "russian",
          "lemma": "мир",
          "grammar_categories": {
            "часть речи": "NOUN",
            "Animacy": "Неодушевленный",
            "падеж": "Винительный падеж",
            "род": "Мужской род",
            "число": "Единственное число"
          }
        },
        {
          "index": 2,
          "token": "!",
          "type": "punct"
        }
      ]
    },
    {
      "tokens": [
        {
          "index": 3,
          "token": "В",
          "type": "russian",
          "lemma": "в",
          "grammar_categories": {
            "часть речи": "ADP"
          }
        },
        {
          "index": 